In [11]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image
from os import listdir
import random
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    normalize])

train_data_list = []
target_list = []
train_data = []
files = listdir('images/train')
for i in range(len(listdir('images/train'))):
    f = random.choice(files)
    files.remove(f)
    img = Image.open("images/train/" + f)
    img_tensor = transform(img)
    
    train_data_list.append(img_tensor)
    cracks = 1 if 'cracks' in f else 0
    nocracks = 1 if 'nocracks' in f else 0
    target = [cracks, nocracks]
    target_list.append(target)
    if len(train_data_list) >= 64:
        train_data.append((torch.stack(train_data_list), target_list))
        train_data_list = []
        print('Loaded batch', len(train_data), 'of', int(len('images/train'))/64)
        print('Percentage Done: ', len (train_data)/int(len(listdir('images/train'))/64), '%')
        if len(train_data)>15:
            break
    
      

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
        self.conv3 = nn.Conv2d(12, 18, kernel_size=5)
        self.conv4 = nn.Conv2d(18, 24, kernel_size=5)
        self.fc1 = nn.Linear(3456, 1000)
        self.fc1 = nn.Linear(1000, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv3(x)
        F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv4(x)
        F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 14112)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.sigmoid(x)
        #print(x.size())
        #exit()
                
model = Net() 
#model.cuda()

optimizer = optim.Adam(model.parameters(), lr=0.01)
def train(epoch):
    model.train()
    batch_id = 0
    for data, target in train_data:
        #data = data.cuda()
        target = torch.Tensor(target) #.cuda()
        data = Variable(data)
        target = Variable(target)
        optimizer.zero_grad()
        out = model(data)
        criterion = F.binary_cross_entropy
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)] \tLoss; {:.6f}'.format(
            epoch, batch_id * len(data), len(train_data.datasets),
            100. * batch_id / len(train_data), loss.data[0]))
        batch_id = batch_id + 1

soma = 0        
def test(): 
    model.eval()
    files = listdir('images/test/')
    f = random.open('images/test/' + f)
    img_eval_tensor = transforms(img)
    img_eval_tensor.unsqueeze(0)
    data = Variable(img_eval_tensor) #img_eval_tensor.cuda()
    out = model(data)
    print(out.data.max(1, keepdim=True)[1])
    img.show()
    global soma
    if soma==0:
        soma = int(input('Soma: '))
        soma=soma
    else:
        x = int(input('Soma: '))
        soma=x+soma
    
total = 0        
for epoch in range(1, 10):
    train(epoch)
    test()
    total=epoch

acuracia = 100*soma/total
print("Soma= ",soma)    
print("Total= ", total)    
print("Acuracia= ", acuracia)
    

AttributeError: module 'random' has no attribute 'open'